In [1]:
!pip install jsonpath-ng requests ipyplot

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import requests
import json
import urllib.request
from jsonpath_ng import jsonpath
from jsonpath_ng.ext import parse
from IPython.display import Image, display, Markdown
import ipyplot

### Get Study

In [3]:
prefix = 'https://www.ebi.ac.uk/biostudies'
study_endpoint = prefix + '/api/v1/studies/{accession}'
accession = 'S-BSST666'

# Load the study json from the API endpoint
url = study_endpoint.format(accession=accession)
print ('Fetching study from {} ...'.format(url))

study = json.loads(requests.get(url).text)
study_attributes = parse('$.section.attributes').find(study)[0].value
attribute_html = list(map( lambda node: '<tr><td><b>{}</b></td><td style="text-align:left">{}</td></tr>'.format(node['name'], node['value']) ,study_attributes ))
display(Markdown('<table>{}</table>'.format("".join(attribute_html))))


Fetching study from https://www.ebi.ac.uk/biostudies/api/v1/studies/S-BIAD4 ...


<table><tr><td><b>Title</b></td><td style="text-align:left">A 3D molecular atlas of the chick embryonic heart</td></tr><tr><td><b>Description</b></td><td style="text-align:left">We present a detailed analysis of gene expression in the 2-day (HH12) embryonic chick heart. RNA-seq of 13 micro-dissected regions reveals regionalised expression of 15,570 genes. Of these, 132 were studied by in situ hybridisation and a subset (38 genes) was mapped by Optical Projection Tomography or serial sectioning to build a detailed 3-dimensional atlas of expression. We display this with a novel interactive 3-D viewer and as stacks of sections, revealing the boundaries of expression domains and regions of overlap. Analysis of the expression domains also defines some sub-regions distinct from those normally recognised by anatomical criteria at this stage of development, such as a previously undescribed subdivision of the atria into two orthogonal sets of domains (dorsoventral and left-right). We also include a detailed comparison of expression in the chick with the mouse and other species.</td></tr><tr><td><b>Study type</b></td><td style="text-align:left">RNAseq</td></tr><tr><td><b>Study Organism</b></td><td style="text-align:left">Gallus gallus</td></tr><tr><td><b>Number of Experiments</b></td><td style="text-align:left">3</td></tr></table>

### Get Image List

In [4]:
# This study has files outside the main JSON in a PageTab attribute is called `File List`
filelists = parse('$..attributes[?(@.name == "File List")]').find(study)

# We will display some images from the first file list here
# Set the caption to the description of the parent section 
description_node = next( filter(lambda node: node['name']=='Description', filelists[0].context.value))
caption = description_node['value']
print (caption)

Histological sections of 40 markers as registered stacks of images


#### Get Image Paths

In [5]:
filename = filelists[0].value['value'] + '.json' 
file_endpoint = prefix + '/files/{accession}/{filename}'
url = file_endpoint.format(accession=accession, filename = filename)
print ('Fetching file list from {}...'.format(url))
filelist = json.loads(requests.get(url).text)
print ('Got {} files'.format( parse('@.`len`').find( filelist)[0].value))

Fetching file list from https://www.ebi.ac.uk/biostudies/files/S-BIAD4/HistologicalSections-LibFile.json...
Got 2418 files


#### Display Images

In [6]:
# Display first 9 images
image_nodes = parse('@[0:9]').find(filelist)
images = []
labels = []
for node in image_nodes:  
    images.append(file_endpoint.format(accession=accession, filename = node.value['path']))
    labels.append( '{} bytes'.format(node.value['size']))
ipyplot.plot_images(images,labels,img_width=300)